In [2]:
urdf = 'C:/Users/USER/Desktop/Chessbot/five_dof_servo_arm_urdf_compatible_description/five_dof_servo_arm_urdf_compatible_description/urdf/five_dof_servo_arm_urdf_compatible.xacro'

# Inverse Kinematics


In [14]:
import pybullet as p
import pybullet_data
import time

# Connect to PyBullet physics simulation with GUI
p.connect(p.GUI)

# Set the path to PyBullet's built-in data
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the robot URDF model (replace 'urdf_file' with the actual URDF file path)
#urdf = "urdf_file"  # Replace with the actual URDF file path
robot_id = p.loadURDF(urdf, useFixedBase=True)

# Optionally, load a plane to give the scene a ground
plane_id = p.loadURDF("plane.urdf")

# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

# Set initial camera position
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])

# Get the number of joints in the robot
num_joints = p.getNumJoints(robot_id)

# Filter out the fixed joints
movable_joints = []
for i in range(num_joints):
    joint_info = p.getJointInfo(robot_id, i)
    print(joint_info)
    joint_type = joint_info[2]
    if joint_type not in [p.JOINT_FIXED]:  # Only add non-fixed joints
        movable_joints.append(i)

# Create sliders for real-time input of end-effector's x, y, z coordinates (IK control)
x_slider = p.addUserDebugParameter("End Effector X", -1, 1, 0)
y_slider = p.addUserDebugParameter("End Effector Y", -1, 1, 0)
z_slider = p.addUserDebugParameter("End Effector Z", 0, 1, 0.5)

# Define end effector link index (adjust based on your URDF's end effector)
end_effector_link_index = 18  # Modify this based on your actual end-effector link index

# Create a visual sphere to represent the target position (where you want the end-effector to go)
target_visual_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[1, 0, 0, 1])  # Red sphere
target_sphere = p.createMultiBody(baseVisualShapeIndex=target_visual_shape)

# Initialize variables for text IDs
target_text_id = None
end_effector_text_id = None
joint_angle_text_ids = [None] * len(movable_joints)  # For joint angle display text IDs

# Main simulation loop
while True:
    # Read the target position from the IK sliders (x, y, z)
    target_x = p.readUserDebugParameter(x_slider)
    target_y = p.readUserDebugParameter(y_slider)
    target_z = p.readUserDebugParameter(z_slider)
    target_position = [target_x, target_y, target_z]
    #target_position = [0.1498, -0.0059,0.2462]
    

    # Move the sphere to the target position
    p.resetBasePositionAndOrientation(target_sphere, target_position, [0, 0, 0, 1])

    # Use inverse kinematics to compute the joint positions for the desired end-effector position
    ik_joint_positions = p.calculateInverseKinematics(robot_id, end_effector_link_index, target_position)
    
    # Apply the calculated joint positions to the movable joints
    for i, joint_index in enumerate(movable_joints):
        p.setJointMotorControl2(robot_id, joint_index, p.POSITION_CONTROL, ik_joint_positions[i])

    # Step the simulation
    p.stepSimulation()
    
    # After applying IK, get the actual position of the end effector (forward kinematics)
    link_state = p.getLinkState(robot_id, end_effector_link_index)
    end_effector_position = link_state[4]  # Extracting the world position (index 4)

    # Display the target and end-effector positions as text on the screen
    target_text = "Target Position: x={:.3f}, y={:.3f}, z={:.3f}".format(target_x, target_y, target_z)
    end_effector_text = "End Effector (FK): x={:.3f}, y={:.3f}, z={:.3f}".format(
        end_effector_position[0], end_effector_position[1], end_effector_position[2])

    # Update the text on screen (create if not initialized)
    if target_text_id is None:
        target_text_id = p.addUserDebugText(target_text, [0, 0, 1.5], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(target_text, [-0.5, -0.50, 1.5], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=target_text_id)

    if end_effector_text_id is None:
        end_effector_text_id = p.addUserDebugText(end_effector_text, [0, 0, 1.3], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(end_effector_text, [-0.5, -0.5, 1.3], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=end_effector_text_id)

    # Display and print joint angles
    for j, joint_index in enumerate(movable_joints):
        joint_angle = p.getJointState(robot_id, joint_index)[0]  # Get the angle of each joint
        joint_angle_text = "Joint {} Angle: {:.3f}".format(joint_index, joint_angle)
        
        # Update joint angle display text on the screen
        if joint_angle_text_ids[j] is None:
            joint_angle_text_ids[j] = p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0)
        else:
            p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0, replaceItemUniqueId=joint_angle_text_ids[j])
        
        # Print the joint angle to the console
        #print(joint_angle_text)

    # Add some delay to slow down the simulation
    time.sleep(1 / 240)  # Simulation step rate (240 Hz real-time)

# Disconnect when done

p.disconnect()


(0, b'rigid_1', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_01_1', (0.0, 0.0, 0.0), (0.0, 0.0, 0.015), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'joint_1', 0, 7, 6, 1, 0.0, 0.0, -1.57, 1.57, 1000.0, 1.0, b'link_11_1', (0.0, 0.0, 1.0), (-0.0010135050049390724, 0.00986527332937008, 0.02780305700191578), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'joint_2', 0, 8, 7, 1, 0.0, 0.0, -1.57, 1.57, 1000.0, 1.0, b'link_21_1', (1.0, 0.0, 0.0), (0.027802718844404835, 0.009865273329370077, 0.0010131668474281253), (0.0, 0.0, 0.0, 1.0), 1)
(3, b'rigid_11', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_21_e_1', (0.0, 0.0, 0.0), (-3.3815752464860793e-07, -5.4655764870556456e-08, 0.022100181267802436), (0.0, 0.0, 0.0, 1.0), 2)
(4, b'rigid_12', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_22_e_1', (0.0, 0.0, 0.0), (-0.025500338157524654, -5.465573751917141e-08, 0.011899818732197559), (0.0, 0.0, 0.0, 1.0), 3)
(5, b'rigid_13', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_22_1', (0.0, 0.0, 0.0), (-3.3815

error: Not connected to physics server.

In [4]:
import pybullet as p
import pybullet_data
import time

# Connect to PyBullet physics simulation with GUI
p.connect(p.GUI)

# Set the path to PyBullet's built-in data
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load the robot URDF model (replace 'urdf_file' with the actual URDF file path)
#urdf = "urdf_file.urdf"  # Replace with the actual URDF file path
robot_id = p.loadURDF(urdf, useFixedBase=True)

# Optionally, load a plane to give the scene a ground
plane_id = p.loadURDF("plane.urdf")

# Set gravity for the simulation
p.setGravity(0, 0, -9.81)

# Set initial camera position
p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=50, cameraPitch=-30, cameraTargetPosition=[0, 0, 0])

# Get the number of joints in the robot
num_joints = p.getNumJoints(robot_id)

# Filter out the fixed joints
movable_joints = []
for i in range(num_joints):
    joint_info = p.getJointInfo(robot_id, i)
    print(joint_info)
    joint_type = joint_info[2]
    if joint_type not in [p.JOINT_FIXED]:  # Only add non-fixed joints
        movable_joints.append(i)

# Create sliders for real-time input of end-effector's x, y, z coordinates (IK control)
x_slider = p.addUserDebugParameter("End Effector X", -1, 1, 0)
y_slider = p.addUserDebugParameter("End Effector Y", -1, 1, 0)
z_slider = p.addUserDebugParameter("End Effector Z", 0, 1, 0.5)

# Create sliders for end-effector orientation (roll, pitch, yaw)
roll_slider = p.addUserDebugParameter("End Effector Roll", -3.14, 3.14, 0)  # Roll in radians
pitch_slider = p.addUserDebugParameter("End Effector Pitch", -3.14, 3.14, 0)  # Pitch in radians
yaw_slider = p.addUserDebugParameter("End Effector Yaw", -3.14, 3.14, 0)  # Yaw in radians

# Define end effector link index (adjust based on your URDF's end effector)
end_effector_link_index = 18  # Modify this based on your actual end-effector link index

# Create a visual sphere to represent the target position
target_visual_shape = p.createVisualShape(p.GEOM_SPHERE, radius=0.01, rgbaColor=[1, 0, 0, 1])  # Red sphere
target_sphere = p.createMultiBody(baseVisualShapeIndex=target_visual_shape)

# Initialize variables for text IDs
target_text_id = None
end_effector_text_id = None
joint_angle_text_ids = [None] * len(movable_joints)  # For joint angle display text IDs

# Main simulation loop
while True:
    # Read the target position from the sliders
    target_x = p.readUserDebugParameter(x_slider)
    target_y = p.readUserDebugParameter(y_slider)
    target_z = p.readUserDebugParameter(z_slider)
    target_position = [target_x, target_y, target_z]

    # Read the target orientation from the sliders
    target_roll = p.readUserDebugParameter(roll_slider)
    target_pitch = p.readUserDebugParameter(pitch_slider)
    target_yaw = p.readUserDebugParameter(yaw_slider)

    # Convert roll, pitch, yaw to quaternion
    target_orientation = p.getQuaternionFromEuler([target_roll, target_pitch, target_yaw])

    # Move the visual sphere to the target position
    p.resetBasePositionAndOrientation(target_sphere, target_position, [0, 0, 0, 1])

    # Use inverse kinematics to calculate joint positions
    ik_joint_positions = p.calculateInverseKinematics(
        robot_id,
        end_effector_link_index,
        target_position,
        targetOrientation=target_orientation
    )

    # Apply the calculated joint positions
    for i, joint_index in enumerate(movable_joints):
        p.setJointMotorControl2(robot_id, joint_index, p.POSITION_CONTROL, ik_joint_positions[i])

    # Step the simulation
    p.stepSimulation()

    # Get the actual end-effector position and orientation
    link_state = p.getLinkState(robot_id, end_effector_link_index)
    end_effector_position = link_state[4]  # World position
    end_effector_orientation = p.getEulerFromQuaternion(link_state[5])  # Convert quaternion to Euler angles

    # Display the target and end-effector position and orientation
    target_text = "Target Position: x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        target_x, target_y, target_z, target_roll, target_pitch, target_yaw
    )
    end_effector_text = "End Effector (FK): x={:.3f}, y={:.3f}, z={:.3f}, roll={:.3f}, pitch={:.3f}, yaw={:.3f}".format(
        end_effector_position[0], end_effector_position[1], end_effector_position[2],
        end_effector_orientation[0], end_effector_orientation[1], end_effector_orientation[2]
    )

    # Display and print joint angles
    for j, joint_index in enumerate(movable_joints):
        joint_angle = p.getJointState(robot_id, joint_index)[0]  # Get the angle of each joint
        joint_angle_text = "Joint {} Angle: {:.3f}".format(joint_index, joint_angle)
        
        # Update joint angle display text on the screen
        if joint_angle_text_ids[j] is None:
            joint_angle_text_ids[j] = p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0)
        else:
            p.addUserDebugText(joint_angle_text, [0, 0, 1.1 - j * 0.1], textColorRGB=[1, 1, 0], textSize=1.0, replaceItemUniqueId=joint_angle_text_ids[j])    
    # Update the displayed text
    if target_text_id is None:
        target_text_id = p.addUserDebugText(target_text, [0, 0, 1.5], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(target_text, [-0.5, -0.50, 1.5], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=target_text_id)

    if end_effector_text_id is None:
        end_effector_text_id = p.addUserDebugText(end_effector_text, [0, 0, 1.3], textColorRGB=[0, 1, 0], textSize=1.2)
    else:
        p.addUserDebugText(end_effector_text, [-0.5, -0.5, 1.3], textColorRGB=[0, 1, 0], textSize=1.2, replaceItemUniqueId=end_effector_text_id)

    # Add some delay to slow down the simulation
    time.sleep(1 / 240)
    



(0, b'rigid_1', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_01_1', (0.0, 0.0, 0.0), (0.0, 0.0, 0.015), (0.0, 0.0, 0.0, 1.0), -1)
(1, b'joint_1', 0, 7, 6, 1, 0.0, 0.0, -1.57, 1.57, 1000.0, 1.0, b'link_11_1', (0.0, 0.0, 1.0), (-0.0010135050049390724, 0.00986527332937008, 0.02780305700191578), (0.0, 0.0, 0.0, 1.0), 0)
(2, b'joint_2', 0, 8, 7, 1, 0.0, 0.0, -1.57, 1.57, 1000.0, 1.0, b'link_21_1', (1.0, 0.0, 0.0), (0.027802718844404835, 0.009865273329370077, 0.0010131668474281253), (0.0, 0.0, 0.0, 1.0), 1)
(3, b'rigid_11', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_21_e_1', (0.0, 0.0, 0.0), (-3.3815752464860793e-07, -5.4655764870556456e-08, 0.022100181267802436), (0.0, 0.0, 0.0, 1.0), 2)
(4, b'rigid_12', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_22_e_1', (0.0, 0.0, 0.0), (-0.025500338157524654, -5.465573751917141e-08, 0.011899818732197559), (0.0, 0.0, 0.0, 1.0), 3)
(5, b'rigid_13', 4, -1, -1, 0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, b'link_22_1', (0.0, 0.0, 0.0), (-3.3815

error: Not connected to physics server.

Grid Spacing 2.8 cm = 0.028 m, roll = -1.157, pitch=yaw =0